Notebook to querry GALEX field and visit information

In [12]:
%matplotlib widget
import matplotlib.pyplot as plt
from astroquery.mast import Observations
from astropy.table import unique, Table
from astropy import units as uu
from astropy.coordinates import SkyCoord
import healpy as hpy
import numpy as np
from vasca.resource_manager import ResourceManager

Get field info from astroquery

In [7]:
#Get TDS field info
tt_tds = Table.read("./TDS-ApJ_766_60/tds_fields_table2.ecsv")
dd_query = {"instrument_name":"GALEX",
             "dataproduct_type":"image",
             "dataRights":"PUBLIC",
             "filters":"NUV",
             "radius":".02 deg",
             "target_name":tt_tds["name"]}
# "coordinates":tt_tds["coordinates"]

tt_obs = Observations.query_criteria(**dd_query)
#tt_obs["obs_id","target_name"].pprint(100)
tt_obs[0:2].show_in_notebook()

Get all-sky visit information

In [8]:
#Get data and add healpix info
rm = ResourceManager()
tt_vis = Table.read(rm.get_path("gal_visits_list", "sas_cloud"))
cosys     = "G" #"Q" for equatorial, "G" for galactic
nside     = 256
npix      = hpy.nside2npix(nside)
tt_vis["hpix"] = hpy.ang2pix(nside,tt_vis["gall"],tt_vis["galb"],lonlat=True,nest=False)

#Show exposure for different surveys
surveys =  ["AIS","MIS","DIS","NGS","GII","CAI"]
for sur in surveys:
    csel = (tt_vis['survey']==sur)
    exp_sum = (tt_vis['nexptime']*csel).sum()
    print(sur,": Visits =",csel.sum(),",exposure =",int(exp_sum),"s ,exposure per visit =",int(exp_sum/csel.sum()),"s")
    
tt_vis[0:2].show_in_notebook()

AIS : Visits = 64375 ,exposure = 6472340 s ,exposure per visit = 100 s
MIS : Visits = 11125 ,exposure = 12199096 s ,exposure per visit = 1096 s
DIS : Visits = 9775 ,exposure = 9791687 s ,exposure per visit = 1001 s
NGS : Visits = 2457 ,exposure = 2793492 s ,exposure per visit = 1136 s
GII : Visits = 7105 ,exposure = 7944605 s ,exposure per visit = 1118 s
CAI : Visits = 6028 ,exposure = 1029497 s ,exposure per visit = 170 s


Define visit selections

In [13]:
#Different selection criteria
sel_all   = tt_vis["hpix"]>-1e6
#"AIS" # "MIS" #"DIS","NGS","GII","CAI"
sel_sur   = (tt_vis['survey'] == "MIS") + (tt_vis['survey'] == "DIS") + (tt_vis['survey'] == "GII")
sel_pid   = tt_vis['ParentImgRunID']== 2552663716076191744
sel_hp    = tt_vis["hpix"] == 8901
sel_tile  = tt_vis["tileNum"] == 5750
obs_pos   = SkyCoord(tt_vis["gall"]*uu.deg, tt_vis["galb"]*uu.deg, frame='galactic')
sel_pos   = SkyCoord(242.36789*uu.deg, 53.67333*uu.deg, frame='icrs') #'galactic'
sel_gal   = obs_pos.separation(sel_pos)<0.5*uu.deg

# Choose selection to apply and print basic info
sel       = sel_sur
tt_sel    = tt_vis[sel]
print(len(tt_sel),"visits in",len(unique(tt_sel, keys='ParentImgRunID')), "fields found")

# Plot visit exposure distribution

28005 visits in 9796 fields found


Plot coverage on the sky

In [10]:
#Setup high res plotting data
nside_hres     = nside*16
npix_hres = hpy.nside2npix(nside_hres)
pix_diam_hres = hpy.nside2resol(nside_hres, arcmin=True) / 60 * uu.deg
print("High resolution pixel diameter:",np.round(pix_diam_hres,4),"Npix:",npix_hres,"Nside:",nside_hres)
tt_sel["hvec"] = hpy.ang2vec(tt_sel["gall"],tt_sel["galb"],lonlat=True)

nr_visits_hres = np.zeros(npix_hres)  # empty map
GALEX_FOV = 0.5  # disc radius in deg

# fill map with coverage data
for vec in tt_sel["hvec"]:
    ipix_disc = hpy.query_disc(nside=nside_hres, vec=vec, radius=np.radians(GALEX_FOV))
    nr_visits_hres[ipix_disc] += 1  # number of visits

#Plot
hpy.mollview(np.log10(nr_visits_hres+1e-3), title="Nr. of visits in log10",nest=False,cmap="nipy_spectral",min=0,max=3,xsize=4800)
hpy.graticule()

#write healpix map to Aladin format, modify names
# hpy.fitsfunc.write_map(filename,[hp_vis, hp_exp],coord="C",column_names=["nr_vis", "exposure"],dtype=[np.float32, np.float32],overwrite=True,partial=True)

High resolution pixel diameter: 0.0143 deg Npix: 201326592 Nside: 4096
